Note: you need to run this as root. Ie `sudo jupyter notebook --allow-root`

In [ ]:
from typing import Tuple, Optional
import struct
import time
import random
import re
import threading
from scapy.all import Ether, ARP, srp, sendp, sniff, get_if_hwaddr, Raw, Packet

To test our code, we'll need to create a virtual switch. To do that, run `setup_script_mac.sh` if you're on mac. If you're not on mac, you brought this upon yourself.

Run this cell to send a message to our bridge and receiving it back.

In [103]:
def send_frame(iface):
    # Create a simple Ethernet frame
    frame = Ether(dst="ff:ff:ff:ff:ff:ff", src=get_if_hwaddr(iface)) / Raw(load="Hello, virtual network!")
    sendp(frame, iface=iface, verbose=False)
    print(f"Sent frame on {iface}")
    
def display_frames(packet):
    frame = packet[0]
    print("Raw bytes: ")
    print(bytes(frame))
    print("Hex: ")
    print(bytes(frame).hex())
    # Scapy will also automatically decode the frame and display it nicely;
    # we won't use that in our router
    print(packet.show())
    


def receive_frame(iface):
    print(f"Listening on {iface}")
    sniff(iface=iface, prn=lambda x: display_frames(x), count=1)
    
def delayed_send(iface):
    time.sleep(5)
    send_frame(iface)


local_interface = "bridge0"
print(f"Using interface: {local_interface}")
# Start a thread to continuously send frames
send_thread = threading.Thread(target=delayed_send, args=(local_interface,))
send_thread.daemon = True
send_thread.start()
receive_frame(local_interface)

Using interface: bridge0
Listening on bridge0
Sent frame on bridge0Raw bytes: 
b'\xff\xff\xff\xff\xff\xff6\t)\xfa\x8c@\x90\x00Hello, virtual network!'
Hex: 
ffffffffffff360929fa8c40900048656c6c6f2c207669727475616c206e6574776f726b21
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 36:09:29:fa:8c:40
  type      = 0x9000
###[ Raw ]### 
     load      = 'Hello, virtual network!'

None



Now we need to make something to turn our wifi into ethernet packets. Run `wifi_to_ethernet_bridge.sh` to do that.

Next, test the wifi bridge.

It should pick up on any packets being sent from your wifi router, and send them to the virtual switch. This means you'll see all the wifi traffic on your router (see if you can spot any websites someone else on the network is visiting!)

In [104]:
internet_interface = "bridge1"
print(f"Using interface: {internet_interface}")
for i in range(5):
    receive_frame(internet_interface)
# Visit a website/general internet traffic

Using interface: bridge1
Listening on bridge1
Raw bytes: 
b"\x01\x00^\x00\x00\xfb\xaeue\x90\xfe\x04\x08\x00E\x00\x01B\xb3\xa9\x00\x00\xff\x11\x1b\xd1\n\n\x00+\xe0\x00\x00\xfb\x14\xe9\x14\xe9\x01.\x1ah\x00\x00\x00\x00\x00\x03\x00\x07\x00\x00\x00\x00\x0f_companion-link\x04_tcp\x05local\x00\x00\x0c\x00\x01\x07_rdlink\xc0\x1c\x00\x0c\x00\x01\x0c_sleep-proxy\x04_udp\xc0!\x00\x0c\x00\x01\xc0\x0c\x00\x0c\x00\x01\x00\x00\x11\x91\x00\x18\x15Jason\xe2\x80\x99s MacBook Pro\xc0\x0c\xc0\x0c\x00\x0c\x00\x01\x00\x00\x11\x91\x00\x16\x13Joe\xe2\x80\x99s MacBook Air\xc0\x0c\xc0\x0c\x00\x0c\x00\x01\x00\x00\x11\x91\x00\x1b\x18Zach\xe2\x80\x99s MacBook Air (2)\xc0\x0c\xc0\x0c\x00\x0c\x00\x01\x00\x00\x11\x91\x00\x16\x13Aryan's MacBook Pro\xc0\x0c\xc0\x0c\x00\x0c\x00\x01\x00\x00\x11\x91\x00\n\x07Asteria\xc0\x0c\xc0\x0c\x00\x0c\x00\x01\x00\x00\x11\x92\x00\x0b\x08arcturus\xc0\x0c\xc0\x0c\x00\x0c\x00\x01\x00\x00\x11\x92\x00\x0c\tMac Clark\xc0\x0c"
Hex: 
01005e0000fbae756590fe04080045000142b3a90000ff111bd10a0a00

In [105]:
local_ip = "192.168.100.2"
internet_ip = "192.168.200.2"
network_prefix = "192.168.100"

Implement a function `create_ethernet_frame` that constructs an Ethernet frame from its components. The function should take four parameters:

1. `dst_mac`: The destination MAC address
2. `src_mac`: The source MAC address
3. `ethertype`: The EtherType field
4. `payload`: The frame's payload data

This is just the reverse of `parse_etehrnet_fame`; you may want to refer back to that.

In [106]:
def create_ethernet_frame(dst_mac: str, src_mac: str, ethertype: int, payload: bytes) -> bytes:
    pass

Write a function `parse_ethernet_frame` that takes a single parameter `frame`. This function should parse an Ethernet frame and return its components. Follow these steps:

1. Extract the destination MAC address:
   - Use the first 6 bytes of the frame
   - Convert it to a hexadecimal string with colons between each byte

2. Extract the source MAC address:
   - Use the next 6 bytes of the frame (bytes 6-12)
   - Convert it to a hexadecimal string with colons between each byte

3. Extract the EtherType:
   - Use the next 2 bytes of the frame (bytes 12-14)
   - Unpack these bytes as a big-endian unsigned short integer

4. Extract the payload:
   - Use all remaining bytes of the frame (from byte 14 onwards)

5. Return the extracted components in the following order:
   destination MAC, source MAC, EtherType, and payload

Notes:
- Use the `bytes.hex()` method with the `':'` separator for MAC addresses
- Use `struct.unpack()` with the format string `'!H'` for the EtherType

In [107]:
def parse_ethernet_frame(frame: bytes) -> Tuple[str, str, int, bytes]:
    pass

Write a function `parse_ip_header` that takes a single parameter `ip_data`. This function should parse an IP header and return its components. Follow these steps:

1. Unpack the first 20 bytes of the IP header:
   - Use `struct.unpack()` with the format string `'!BBHHHBBH4s4s'`
   - Store the result in a variable named `iph`

2. Extract the version and IHL (Internet Header Length):
   - The first byte contains both version and IHL
   - Extract version by right-shifting the first byte by 4 bits
   - Extract IHL by masking the first byte with 0xF and multiplying by 4

3. Extract the TTL (Time To Live):
   - This is the 6th byte in the header (index 5 in `iph`)

4. Extract the protocol:
   - This is the 7th byte in the header (index 6 in `iph`)

5. Extract and convert source IP address:
   - Use the 9th 4-byte chunk from `iph` (index 8)
   - Convert from network byte order to a string using `socket.inet_ntoa()`

6. Extract and convert destination IP address:
   - Use the 10th 4-byte chunk from `iph` (index 9)
   - Convert from network byte order to a string using `socket.inet_ntoa()`

7. Extract the payload:
   - This is all data after the header (use `ip_data[ihl:]`)

8. Return all extracted components in the following order:
   version, IHL, TTL, protocol, source IP, destination IP, and payload

Notes:
- Import necessary modules (you'll need `struct` and `socket`)
- The `inet_ntoa()` function converts an IP address from 32-bit packed binary format to a string

In [108]:
def parse_ip_header(ip_data: bytes) -> Tuple[int, int, int, int, str, str, bytes]:
    pass

We've parsed the header of our IP packet, but the IP protocol only tells us a source and destination IP address. It doesn't let us specify a port. Instead, the data in the IP packet contains the port information.

There are [many](https://www.iana.org/assignments/protocol-numbers/protocol-numbers.xhtml) different standards for the packet data, but we'll only support the most common two: [TCP](https://en.wikipedia.org/wiki/Transmission_Control_Protocol#TCP_segment_structure) and [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol#UDP_datagram_structure)

TCP and UDP packets have their own header, which is where the port is specified. In both protocols, the first byte is the source port and the second byte is the destination port. Since that's all we care about, **we can treat both protocols identically**.

Write a function to parse the source and destination ports from IP packets. The function should take two parameters: `data` and `protocol`.

Implement the following logic in your function:
   
1. Check if the `protocol` is either 6 (TCP) or 17 (UDP). If it's not, return return `None, None`.

2. If the protocol matches:
   - Use `struct.unpack()` to extract two unsigned short integers (16-bit) from the first 4 bytes of `data`.
   - Use the format string '!HH' for network byte order (big-endian).
   - Assign the unpacked values to `src_port` and `dst_port`.
   - Return `src_port` and `dst_port` as a tuple.

Notes:
- If we wanted to properly handle TCP packets, we would need to update the checksum at the end of the packet (TCP requires a _separate_ checksum from the IP checksum). However, we're not going to do that here. UDP also has a checksum, but it's optional in IPv4, so it's less egregious that we're ignoring it.

In [109]:
def parse_tcp_udp_header(data: bytes, protocol: int) -> Tuple[Optional[int], Optional[int]]:
    pass

# IP Header Creation

Given a source and destination IP, create an IP header. The IP header is some data at the start of the packet that contains routing information. It's formatted as per [the IPv4 standard](https://en.wikipedia.org/wiki/IPv4#Header)

## Header Structure

1. **Version (4 bits)**: 4 for IPv4
2. **Header Length (4 bits)**: Fixed length of 5 (no options)
3. **Type of Service (1 byte)**:
   - Set to 0 (default)
   - Options can be found at https://en.wikipedia.org/wiki/Differentiated_services
   - Includes Explicit Congestion Notification (also set to 0)
4. **Total Length (2 bytes)**: Length of the entire packet (header + data)
5. **Identification (2 bytes)**: Set to a random number
6. **Fragmentation (2 bytes)**: Not used in this case
7. **Time To Live (TTL) (1 byte)**: Decremented each time the packet is forwarded
8. **Protocol (1 byte)**:
   - Set to 6 for TCP
   - Full list: https://en.wikipedia.org/wiki/List_of_IP_protocol_numbers
9. **Header Checksum (2 bytes)**: Set to 0 for now, to be calculated based on the header

## Question

1. What is the maximum possible length of data that can be sent in a single packet, excluding the header, based on what we know so far?

In [110]:
def create_ip_header(src_ip: str, dst_ip: str, ttl: int, protocol: int, total_length: int) -> bytes:
    pass

Implement a function `calculate_checksum(data)` that computes the Internet Checksum for the given packet.

> The checksum field is the 16 bit one's complement of the one's complement sum of all 16 bit words in the header. For purposes of computing the checksum, the value of the checksum field is zero.

You can read more about it on [Wikipedia](https://en.wikipedia.org/wiki/Internet_checksum)

This checksum is commonly used in network protocols like IP, TCP, and UDP. Follow these general steps:

1. Ensure the data has an even number of bytes, padding if necessary.

2. Convert the byte sequence into a series of 16-bit integers.

3. Add all the 16-bit words together.

4. If there's any overflow beyond 16 bits during addition, add the overflow back to the sum.

5. Take the [one's complement](https://en.wikipedia.org/wiki/Ones%27_complement) of the final sum. (AKA flip all the bits)

6. The checksum should be a 16-bit value.

Question: if there's no corruption, what should the sum of the whole IP header, including the checksum, represented as a 16-bit int, be?

In [111]:
def calculate_checksum(data: bytes) -> int:
    pass

In [112]:
# Routing table to keep track of which interface to use for a given destination
routing_table = {}

def update_routing_table(mac_address, interface):
    routing_table[mac_address] = interface

Implement a function `handle_nat` that performs Network Address Translation (NAT) for outgoing packets. The function should take the following parameters:

1. `protocol`: The protocl of the packet (e.g., 'TCP', 'UDP')
2. `src_ip`: Original source IP address
3. `src_port`: Original source port
4. `dst_ip`: Destination IP address
5. `dst_port`: Destination port

The function should perform the following steps:

1. Connection Identification:
   - Create a unique identifier for the connection using the protocol, source IP, and source port.
   - Log or print the NAT operation details for debugging purposes.

2. NAT Table Lookup:
   - Check if the connection identifier exists in the NAT table.

3. New Connection Handling:
   - If the connection is new (not in the NAT table):
     a. Generate a new 'public' port number from the dynamic/private port range (49152-65535).
     b. Create a new entry in the NAT table mapping the connection ID to the new public port.
     c. Create a reverse mapping in the NAT table for incoming packets.

4. Return Translation:
   - Return the public IP address (internet-facing IP) and the translated port number.

Notes:
- The function should handle potential port conflicts and ensure unique mappings.

In [113]:
# NAT table to keep track of connections

nat_table = {}

def handle_nat(protocol: str, src_ip: str, src_port: int, dst_ip: str, dst_port: int) -> Tuple[str, int]:
   pass

Here's a utility function to forward a frame and log it. It might come in handy.

In [114]:
def forward_frame(frame, output_interface):
    sendp(Raw(frame), iface=output_interface, verbose=False)
    print(f"Forwarded frame via {output_interface}")

In [115]:
def modify_ip_header(ip_header, new_src_ip, new_dst_ip, new_ttl):
    new_ip_header = create_ip_header(new_src_ip, new_dst_ip, ip_header[6], len(ip_header), new_ttl)
    checksum = calculate_checksum(new_ip_header)
    return new_ip_header[:10] + struct.pack('!H', checksum) + new_ip_header[12:]

In [116]:
def modify_tcp_udp_header(header, new_src_port=None, new_dst_port=None):
    src_port, dst_port = struct.unpack('!HH', header[:4])
    if new_src_port is not None:
        src_port = new_src_port
    if new_dst_port is not None:
        dst_port = new_dst_port
    return struct.pack('!HH', src_port, dst_port) + header[4:]

Implement a function `handle_outgoing_packet` that processes an outgoing network packet. The function should take the following parameters:

1. `src_ip`: Source IP address
2. `src_port`: Source port
3. `dst_ip`: Destination IP address
4. `dst_port`: Destination port
5. `protocol`: Network protocol (e.g., TCP, UDP)
6. `ip_header`: The original IP header
7. `ip_payload`: The original IP payload (including transport layer header and data)

The function should perform the following steps:

1. Apply NAT to potentially modify the source IP and port.

2. Decrement the TTL value from the IP header.

3. Update the IP header with the new source IP (post-NAT) and TTL.

4. Update the transport layer header (TCP or UDP) with the new source port (post-NAT).

5. Combine the modified IP header and transport layer header.

6. Determine the destination MAC address using a routing table.
   - If not found, use a broadcast MAC address.

7. Get the MAC address of the outgoing network interface.

8. Construct an Ethernet frame using the resolved MAC addresses, appropriate EtherType for IP (0x0800), and the modified payload.

9. Return the created Ethernet frame and the identifier for the outgoing network interface.

In [117]:
def handle_outgoing_packet(src_ip: str, src_port: int, dst_ip: str, dst_port: int, protocol: str, ip_header: bytes, ip_payload: bytes) -> Tuple[bytes, str]:
    pass

Now see if you can write logic to handle incoming packets.

If the nat table has an entry for the incoming packet, translate the destination IP and port to the original IP and port. Then, construct an Ethernet frame with the translated IP and port, and return it.

Can you figure out everything else you might need to do to handle incoming packets?

In [118]:
def handle_incoming_packet(src_ip: str, dst_ip: str, dst_port: int, protocol: str, ip_header: bytes, ip_payload: bytes) -> Tuple[bytes, str]:
    pass

# Handling Incoming Packets!

You've got all the tools you need, go ahead and implement the logic to handle incoming packets!

In [119]:
def packet_handler(packet: Packet):
    frame = bytes(packet)
    pass

This function will listen on all the interfaces we pass it in the `iface` parameter. It will then send any incoming Ethernet frames to the function passed to `prn`. `prn` is just a callback that will be called with the Ethernet frame as a parameter. IMO it's poorly named.

In [120]:
def start_router():
    print("Router started. Listening for frames...")
    sniff(iface=[internet_interface, local_interface], prn=packet_handler, store=0)

In [122]:
start_router()

Router started. Listening for frames...


To test your router, run `sudo python send_test_packet.py` to send a test packet to the router. You should see the router log how it's handling the packet.

Challenges: Try to make your router...
1. Block traffic to a blacklist of IP addresses.
2. Maliciously re-route traffic from google.com to https://elgoog.im/
3. Log the mac addresses of anyone who tries to visit https://thepiratebay.org/